## Rating csv


In [1]:
import pandas as pd
import datetime
import numpy as  np
from tqdm import tqdm
np.random.seed(1337)
with open('./kaggle/rating_train.csv', 'r') as f:
    ls = f.readlines()[1:]
u_map = {}

dates = []
foods = []
users = []
    

with tqdm(total=len(ls)) as pbar:
    for l in ls:
        date_str, user, food = l.strip().split(',')
        date = datetime.datetime.strptime(date_str, '%Y-%m-%d')
        user, food = int(user), int(food)
        if user not in u_map:
            u_map[user] = []
        u_map[user].append( (date, food) )
        
        dates.append(date)
        foods.append(food)
        users.append(user)
        pbar.update(1)
        

        
food_map = {f:i for i, f in enumerate(set(foods))}
user_map = {u:i for i, u in enumerate(set(users))}



100%|██████████| 2681494/2681494 [00:25<00:00, 105526.64it/s]


In [3]:
import numpy as np
a = []
b = []

for k,v in u_map.items():
    ds = [d for d,f in v]
    
    set_ds = set(ds)
    a.append(len(set_ds))
    b.append(len(ds))
print max(a), min(a),np.mean(a), np.std(a)
print max(b), min(b),np.mean(b), np.std(b)

160 23 95.21165644171779 34.48409857182605
5569 194 1028.1802147239264 599.6642539977294


## User csv


In [ ]:
import pandas as pd
import numpy as np
import keras
import os
from constants import MAX_TEXT_SEQ_LEN, MAX_NUM_WORDS, EMBEDDING_DIM
from keras.preprocessing.sequence import pad_sequences



csv = pd.read_csv('./kaggle/user.csv')
print csv.columns
# print 'userid,username,age,gender,location,city,state,title,about_me,reasons,inspirations,friends_count'
# for csv.iterrows
texts = []
print 'Starting read texts.'
def append_text(texts, value):
    if not pd.isnull(value):
        texts.append(value)
for row in csv.iterrows():
    r = row[1]
    append_text(texts, r['about_me'])
    append_text(texts, r['reasons'])
    append_text(texts, r['inspirations'])

n = csv['about_me'][2580]
print np.isnan(n)

buf = [len(s) for s in texts]
print np.max(buf), np.mean(buf), np.std(buf)

tokenizer = keras.preprocessing.text.Tokenizer(num_words=MAX_TEXT_SEQ_LEN, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~ ',
                                   lower=True, split=' ', char_level=False, oov_token=None)
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
sequences = tokenizer.texts_to_sequences(texts)
data = pad_sequences(sequences, maxlen=MAX_TEXT_SEQ_LEN)


print('Preparing embedding matrix.')
# embeddings_index = {}
# # with open(os.path.join('./','glove', 'glove.6B.%dd.txt' % EMBEDDING_DIM), 'r', encoding='utf8') as f:
# with open(os.path.join('./','glove', 'glove.6B.%dd.txt' % EMBEDDING_DIM), 'r') as f:
#     for line in f:
#         values = line.split()
#         word = values[0]
#         coefs = np.asarray(values[1:], dtype='float32')
#         embeddings_index[word] = coefs
# # prepare embedding matrix
# num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
# embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
# for word, i in word_index.items():
#     if i > MAX_NUM_WORDS:
#         continue
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         # words not found in embedding index will be all-zeros.
#         embedding_matrix[i] = embedding_vector

print('done')

In [2]:
n = csv['about_me'][2580]
print pd.isnull(n)

True


### Generate training data and label

#### prepare sample generator


In [2]:
from constants import MAX_SEQ_LEN


idx = np.random.permutation(len(u_map))
val_num = len(u_map)//10
train_idx, val_idx = idx[val_num:], idx[:val_num]
train_u_map = {k:u_map[k] for k in u_map.keys()[val_num:]}
val_u_map = {k:u_map[k] for k in u_map.keys()[:val_num]}
def batch_boostrap_generator(batch_size, u_map, food_map, max_history_len):
    G = boostrap_generator(u_map, food_map, max_history_len)
    while True:
        X = []
        for i in range(batch_size):
            x = next(G)
            x = np.pad(x, ((0,max_history_len-x.shape[0]),(0,0)), 'constant', constant_values=0)
            X.append(np.expand_dims(x, axis=0))
        yield np.vstack(X)
def boostrap_generator(u_map, food_map, max_history_len):
    while True:
        keys = u_map.keys()
        for user_idx in np.random.permutation(len(u_map)):
            user = keys[user_idx]
            X = np.zeros([max_history_len, len(food_map)])
            history = u_map[user]
            ds = np.array([d for d,f in history])
            fs = np.array([f for d,f in history])
            sorted_idx = np.argsort(ds)
            ds = ds[sorted_idx]
            fd = fs[sorted_idx]
            
            date_idx = 0
            now_date = ds[0]
            for food, date in zip(fs,ds):
                if date != now_date:
                    date_idx+=1
                    now_date = date
                X[date_idx, food_map[food]] = 1
            
            yield X
            
    

G = batch_boostrap_generator(32, train_u_map, food_map, max_history_len=MAX_SEQ_LEN)
val_G = batch_boostrap_generator(32//2, val_u_map, food_map, max_history_len=MAX_SEQ_LEN)

x = next(G)
print x.shape
x = next(val_G)
print x.shape

G2 = boostrap_generator(train_u_map, food_map, max_history_len=MAX_SEQ_LEN)
x = next(G2)
print x.shape


(32, 165, 5532)
(16, 165, 5532)
(165, 5532)


### RNN model with one-for-all BCE loss

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F### Transformer with ALS embedding Training
# import Transformer/

import numpy as np
from constants import FOOD_NUM, USER_NUM
class Net(nn.Module):

    def __init__(self, dm, p_drop):
        super(Net, self).__init__()
        self.drop = nn.Dropout(p_drop)
        self.food_emb = Food_embedding(FOOD_NUM, dm, 2, p_drop)
        self.rnn_lay_num = 2
        self.rnn = nn.GRU(dm, dm, self.rnn_lay_num)
        self.output_linear = nn.Linear(dm, FOOD_NUM)

    def forward(self, history):
#         print(K.size(), get_pos_mat(MAX_SEQUENCE_LENGTH).size())
        
        x = self.food_emb(history)
        batch, x_len, d = x.size()
        
        x = x.permute(1,0,2)
        h0 = torch.zeros([self.rnn_lay_num, batch, d], requires_grad=False).cuda()
        rnn_out, hn = self.rnn(x, h0)
        
        x = rnn_out.permute(1,0,2)
        x = self.output_linear(x)
        x = torch.sigmoid(x)
        return x
        
class Food_embedding(nn.Module):
    def __init__(self, c_in, dm, layer_num, p_drop, activation_fn=F.selu):
        super(Food_embedding, self).__init__()
        self.activation_fn = activation_fn
        self.drop = nn.Dropout(p_drop)
        assert layer_num >= 1
        self.first_linear = nn.Linear(c_in, dm)
        self.linears = nn.ModuleList([nn.Linear(dm, dm) for i in range(layer_num-1)])
        

    def forward(self, x):
#         print(K.size(), get_pos_mat(MAX_SEQUENCE_LENGTH).size())
        x = self.first_linear(x)
#         x = self.activation_fn(x)
        x = self.drop(x)
        for lay in self.linears:
            x = self.drop(self.activation_fn(lay(x)))
        return x
    
batch = 7
dm = 128
K = torch.rand([batch, 18, FOOD_NUM]).cuda()
model = Net(dm, 0.1).cuda()
o = model(K)
# print t
print(o.size())
# print o

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))




/home/k123/env/python2.7.15/lib/python2.7/site-packages/torch/backends/cudnn/__init__.py:102: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "


torch.Size([7, 18, 5532])
1636508


/home/k123/env/python2.7.15/lib/python2.7/site-packages/torch/nn/modules/rnn.py:179: RuntimeWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  self.dropout, self.training, self.bidirectional, self.batch_first)


In [4]:
from collections import deque
from tqdm import tqdm as tqdm

import time
def dump_log(model, n_iter, loss, acc, val_loss, val_acc, log_file_stream, tmp_model_path):
    log_text = '%.7d<split>%.5f<split>%.5f<split>%.5f<split>%.5f\n' % (n_iter, loss, acc, val_loss, val_acc)
    log_file_stream.write(log_text)
    if n_iter % 10 == 0 :
        log_file_stream.flush()
        torch.save(model, tmp_model_path)
def normal_acc(pred, label):
    mask = label.type(torch.uint8)
    buf = pred == mask
    buf = buf.masked_select(mask)
    
    acc = torch.sum(buf).item() / float(torch.sum(mask).item())
    return acc
def rev_mask(m):
    out = torch.ones_like(m, dtype=torch.uint8, requires_grad=False)
    out.masked_fill_(m, 0)
    return out
        
acc_q = deque(maxlen=10000)
loss_q = deque(maxlen=10000)
val_acc_q = deque(maxlen=10000)
val_loss_q = deque(maxlen=10000)
t = time.time()
best_acc  = 0
best_loss = float('inf')

epochs = 100
batch_size = 32
G = batch_boostrap_generator(batch_size, train_u_map, food_map, max_history_len=MAX_SEQ_LEN)
val_G = batch_boostrap_generator(batch_size, val_u_map, food_map, max_history_len=MAX_SEQ_LEN)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
print 'start training.'
with open('log2.txt', 'a') as f:
    with open('best2.txt', 'w') as best_log:
        iters = 100000000
        with tqdm(total=iters) as pbar:
            for it in range(iters):
                optimizer.zero_grad()
                model.train()
                seq = next(G)
                seq = torch.FloatTensor(seq).cuda()
                seq.requires_grad_(False)

                x = seq[:,:-1,:]
                y = seq[:,1:,:]

                output = model(x)
#                     a,b = foo(output, y)
#                     loss = normal_loss(criterion, output, y)
                loss = criterion(output, y)
                pred = output > 0.5
                label = y


                acc = normal_acc(pred, label)
                acc_q.append(acc)
                loss.backward()

                optimizer.step()
                with torch.no_grad():
                    model.eval()
                    seq = next(val_G)
                    seq = torch.FloatTensor(seq).cuda()
                    seq.requires_grad_(False)

                    x = seq[:,:-1,:]
                    y = seq[:,1:,:]
                    output = model(x)
#                     c,d = foo(output, y)
#                         val_loss = normal_loss(criterion, output, y)
#                         val_loss = criterion(output, y)
#                         pred = output > 0.5
                    val_loss = criterion(output, y)
                    pred = output > 0.5

                    label = y

                    val_acc = normal_acc(pred, label)
                    val_acc_q.append(val_acc)
                acc = np.mean(acc_q)
                val_acc = np.mean(val_acc_q)

#                     pbar.set_postfix_str('acc : %.3f, val_acc : %.3f, loss : %.3f, val_loss : %.3f \t %.3f, %.3f, %.3f, %.3f' % (acc, val_acc, loss.item(), val_loss.item(), a,b,c,d), refresh=False)
                pbar.set_postfix_str('acc : %.3f, val_acc : %.3f, loss : %.3f, val_loss : %.3f' % (acc, val_acc, loss.item()*1000., val_loss.item()*1000.), refresh=False)
                pbar.update(batch_size)
                dump_log(model, (it+1)*batch_size, loss, acc, val_loss, val_acc, f, './tmp2.pt')
                if val_acc > best_acc and it > 100:
                    torch.save(model, './best2.pt')
                    best_acc = val_acc
                    best_log.write('%d\t%.5f\n' % ((it+1)*batch_size, best_acc))

# Train model
print("Optimization Finished!")
# print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

  0%|          | 0/100000000 [00:00<?, ?it/s]

start training.


  0%|          | 160/100000000 [00:05<984:18:28, 28.22it/s, acc : 0.416, val_acc : 0.343, loss : 663.147, val_loss : 644.682] /home/k123/env/python2.7.15/lib/python2.7/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/k123/env/python2.7.15/lib/python2.7/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Food_embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
  4%|▎         | 3749792/100000000 [23:47:01<619:55:06, 43.13it/s, acc : 0.188, val_acc : 0.085, loss : 3.697, val_loss : 5.998]


KeyboardInterrupt: 

### Transformer with ALS embedding Training

In [4]:
### Transformer with ALS embedding Trainingimport torch
import torch.nn as nn
import torch.nn.functional as F### Transformer with ALS embedding Training
# import Transformer
import numpy as np
from constants import FOOD_NUM, USER_NUM
class Net(nn.Module):

    def __init__(self, dm, p_drop):
        super(Net, self).__init__()
        self.drop = nn.Dropout(p_drop)
        self.food_emb = Food_embedding(FOOD_NUM, dm, 2, p_drop)
        self.summary_weight = nn.Parameter(torch.FloatTensor(1, dm))
        nn.init.xavier_normal_(self.summary_weight)
        self.linears = nn.ModuleList([nn.Linear(dm, dm) for i in range(2)])
        self.output_linear = nn.Linear(dm, FOOD_NUM)

    def forward(self, history):
#         print(K.size(), get_pos_mat(MAX_SEQUENCE_LENGTH).size())
        
        K = history
        
        K = self.food_emb(K)
        
        
        batch, K_len, d = K.size()
        Q = self.summary_weight.repeat(batch,1).view(batch, 1, -1)
        x = att_out = self.attention(Q, K)
        for lay in self.linears:
            x = F.selu(lay(x))
            x = self.drop(x)
        x = self.output_linear(x)
        return x
    def to_ont_hot(self, one_hot_dim, indices):
        batch, _ = indices.size()
        one_hot = torch.zeros(batch, one_hot_dim, requires_grad=False)
        one_hot.scatter_(1, indices.cpu(), 1)
        return one_hot.cuda()
    def attention(self, Q,K):
        assert len(Q.shape) == 3 and len(K.shape) == 3
        e = torch.bmm(Q,K.permute(0,2,1))
        
        att = F.softmax(e, dim=-1)
        out = torch.bmm(att, K).squeeze(1)
        return out
        
class Food_embedding(nn.Module):
    def __init__(self, c_in, dm, layer_num, p_drop, activation_fn=F.selu):
        super(Food_embedding, self).__init__()
        self.activation_fn = activation_fn
        self.drop = nn.Dropout(p_drop)
        assert layer_num >= 1
        self.first_linear = nn.Linear(c_in, dm)
        self.linears = nn.ModuleList([nn.Linear(dm, dm) for i in range(layer_num-1)])
        

    def forward(self, x):
#         print(K.size(), get_pos_mat(MAX_SEQUENCE_LENGTH).size())
        x = self.first_linear(x)
        x = self.activation_fn(x)
        x = self.drop(x)
        for lay in self.linears:
            x = self.drop(self.activation_fn(lay(x)))
        return x
    
batch = 7
dm = 128
K = torch.rand([batch, 18, FOOD_NUM]).cuda()
model = Net(dm, 0.1).cuda()
o = model(K)
# print t
print(o.size())
# print o

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))




torch.Size([7, 5532])
1471516


In [6]:
from collections import deque
from tqdm import tqdm as tqdm

import time
def dump_log(model, n_iter, loss, acc, val_loss, val_acc, log_file_stream, tmp_model_path):
    log_text = '%.7d<split>%.5f<split>%.5f<split>%.5f<split>%.5f\n' % (n_iter, loss, acc, val_loss, val_acc)
    log_file_stream.write(log_text)
    if n_iter % 10 == 0 :
        log_file_stream.flush()
        torch.save(model, tmp_model_path)
def normal_acc(pred, label):
    mask = label.type(torch.uint8)
    buf = pred == mask
    buf = buf.masked_select(mask)
    
    acc = torch.sum(buf).item() / float(torch.sum(mask).item())
    return acc
def rev_mask(m):
    out = torch.ones_like(m, dtype=torch.uint8, requires_grad=False)
    out.masked_fill_(m, 0)
    return out
def normal_loss(criterion, output, y):
    loss = criterion(output*10., y*10.)
    label = y.type(torch.uint8)
    
    
    dim = label.shape[-1]
    sum_1s = torch.sum(label)
    loss_mask = y.clone()*(dim-sum_1s)

    loss_mask.masked_fill_(rev_mask(label), sum_1s)
    return torch.mean(loss_mask*loss)
def foo(output, y):
    output = output[0,:].flatten()
    y = y[0,:].flatten()
    pos_i = neg_i = -1
    for i in np.random.permutation(len(y)):
        if y[i] == 1 and pos_i == -1:
            pos_i = i
        if y[i] == 0 and neg_i == -1:
            neg_i = i
        if pos_i != -1 and neg_i != -1:
            break
    return output[pos_i], output[neg_i]
        
acc_q = deque(maxlen=1000)
loss_q = deque(maxlen=1000)
val_acc_q = deque(maxlen=1000)
val_loss_q = deque(maxlen=1000)
t = time.time()
best_acc  = 0
best_loss = float('inf')

epochs = 100
batch_size = 64
G = batch_boostrap_generator(batch_size, train_u_map, food_map, Y_map, max_history_len=MAX_SEQ_LEN, flip=False)
val_G = batch_boostrap_generator(batch_size, val_u_map, food_map, Y_map, max_history_len=MAX_SEQ_LEN, flip=False)
criterion = nn.MSELoss(reduction='none')
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
print 'start training.'
with open('log.txt', 'a') as f:
    with open('best.txt', 'w') as best_log:
        for e in range(epochs):
            iters = len(food_map) * len(user_map)
            with tqdm(total=iters) as pbar:
                for it in range(iters):
                    optimizer.zero_grad()
                    model.train()
                    x, y = next(G)
    #                 print 'x1', np.sum(x[0,:])
                    x = torch.FloatTensor(x).cuda()
                    y = torch.FloatTensor(y).cuda()
                    x.requires_grad_(False)
                    y.requires_grad_(False)

                    output = model(x)
                    a,b = foo(output, y)
                    loss = normal_loss(criterion, output, y)
                    pred = output > 0.5
                    label = y
                    

                    acc = normal_acc(pred, label)
                    acc_q.append(acc)
                    loss.backward()
                    
                    optimizer.step()
                    with torch.no_grad():
                        model.eval()
                        x, y = next(val_G)
    #                     print 'x2', np.sum(x[0,:])
                        x = torch.FloatTensor(x).cuda()
                        y = torch.FloatTensor(y).cuda()
                        x.requires_grad_(False)
                        y.requires_grad_(False)
                        output = model(x)
                        c,d = foo(output, y)
                    
                        val_loss = normal_loss(criterion, output, y)
                        pred = output > 0.5
                        label = y

                        val_acc = normal_acc(pred, label)
                        val_acc_q.append(val_acc)
                    acc = np.mean(acc_q)
                    val_acc = np.mean(val_acc_q)

                    pbar.set_postfix_str('acc : %.3f, val_acc : %.3f, loss : %.3f, val_loss : %.3f \t %.3f, %.3f, %.3f, %.3f' % (acc, val_acc, loss.item(), val_loss.item(), a,b,c,d), refresh=False)
                    pbar.update(batch_size)
                    dump_log(model, (it+1)*batch_size, loss, acc, val_loss, val_acc, f, './tmp.pt')
                    if val_acc > best_acc and it > 100:
                        torch.save(model, './best.pt')
                        best_acc = val_acc
                        best_log.write('%d\t%.5f\n' % ((it+1)*batch_size, best_acc))

# Train model
print("Optimization Finished!")
# print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

  0%|          | 0/14427456 [00:00<?, ?it/s]

start training.


  0%|          | 320/14427456 [00:05<70:08:44, 57.13it/s, acc : 0.000, val_acc : 0.000, loss : 2622.345, val_loss : 2432.507 	 0.017, 0.084, -0.013, -0.001] /home/k123/env/python2.7.15/lib/python2.7/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/k123/env/python2.7.15/lib/python2.7/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Food_embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
  4%|▎         | 507136/14427456 [2:46:27<115:52:09, 33.37it/s, acc : 0.401, val_acc : 0.154, loss : 1218.665, val_loss : 2220.557 	 0.063, 0.005, 0.206, 0.015]   


KeyboardInterrupt: 

In [59]:
a = torch.FloatTensor([0,0,1,1,0,0])
torch.

tensor([0, 0, 1, 1, 0, 0], dtype=torch.uint8)
tensor([0, 0, 0, 1, 0, 1], dtype=torch.uint8)
tensor([0., 0., 1., 0., 0., 1.])
tensor([2., 2., 2., 4., 2., 4.])
tensor(1.)


In [68]:
a = torch.FloatTensor([0,0,1,1,0,0])
b = torch.ByteTensor([0,0,0,1,0,1])
a.requires_grad_(False)
print a.clone().requires_grad

False


### testing with RNN model

In [5]:
from constants import MAX_SEQ_LEN

model = torch.load('./best.pt')
model = model.cuda()
model.eval()
rev_food_map = {v:k for k,v in food_map.items()}
with torch.no_grad():
    with open('predict.csv', 'w') as f_out:
        f_out.write('userid,foodid\n')

        with tqdm(total=len(u_map)) as pbar:
            for user in u_map.keys():
                x = np.zeros([MAX_SEQ_LEN, len(food_map)])
                history = u_map[user]
                ds = np.array([d for d,f in history])
                fs = np.array([f for d,f in history])
                sorted_idx = np.argsort(ds)
                ds = ds[sorted_idx]
                fd = fs[sorted_idx]

                date_idx = 0
                now_date = ds[0]
                for food, date in zip(fs,ds):
                    if date != now_date:
                        date_idx+=1
                        now_date = date
                    x[date_idx, food_map[food]] = 1
                x = torch.FloatTensor(x).unsqueeze(0).cuda()
                out = model(x)

                arr = out.cpu().numpy()[0,-1,:]
                k20 = reversed(np.argsort(arr)[-20:])
                s = ''
                for food_idx in k20:
                    s += ' %d' % rev_food_map[food_idx]
                f_out.write('%d,%s\n' % (user, s) )

                pbar.update(1)
                

print 'done'

    
            


100%|██████████| 2608/2608 [00:21<00:00, 120.19it/s]

done


### testing with Transformer model

In [ ]:
from constants import MAX_SEQ_LEN

def batch_boostrap_generator(batch_size, u_map, food_map, Y_map, max_history_len, flip):
    G = boostrap_generator(u_map, food_map, Y_map, max_history_len, flip)
    while True:
        X = []
        pos_Y = []
        neg_Y = []
        for i in range(batch_size):
            x, pos_y, neg_y = next(G)
            x = np.pad(x, ((0,max_history_len-x.shape[0]),(0,0)), 'constant', constant_values=0)
            X.append(np.expand_dims(x, axis=0))
            pos_Y.append(np.expand_dims(pos_y, axis=0))
            neg_Y.append(np.expand_dims(neg_y, axis=0))
        yield np.vstack(X), np.vstack(pos_Y), np.vstack(neg_Y) 

        
flip = True       
model = torch.load('./best.pt')
model.eval()
batch_size = 256
food_buf = torch.LongTensor(np.arange(len(food_map))).view(1,-1).repeat(batch_size, 1).cuda()
answer_sheet = np.zeros([len(u_map), len(food_map)])

with torch.no_grad():
    with tqdm(total=len(u_map)*len(food_map)) as pbar:
        keys = u_map.keys()
        ran = range((len(keys) // batch_size) +1) if len(keys) % batch_size != 0 else range(len(keys) // batch_size)
        for i in ran:
            a = i*batch_size
            b = (i+1)*batch_size if (i+1)*batch_size < len(keys) else len(keys)
            X = []
            for user in keys[a:b]:
                x = np.zeros([MAX_SEQ_LEN, len(food_map)])
                history = u_map[user]
                ds = np.array([d for d,f in history])
                fs = np.array([f for d,f in history])
                sorted_idx = np.flip(np.argsort(ds), axis=-1) if flip else np.argsort(ds)
                ds = ds[sorted_idx]
                fd = fs[sorted_idx]

                date_idx = 0
                now_date = ds[0]
                for food, date in zip(fs,ds):
                    if date != now_date:
                        date_idx+=1
                        now_date = date
                    x[date_idx, food_map[food]] = 1
                X.append(np.expand_dims(x,axis=0))
            
            X = torch.FloatTensor(np.vstack(X)).cuda()
            for food_idx  in range(len(food_map)):
                target = food_buf[:b-a, food_idx:food_idx+1]
                output = model(target, X)
                answer_sheet[a:b, food_idx:food_idx+1] =  output.cpu()
                pbar.update(batch_size)
                
np.save('./output_sheet', answer_sheet)
print answer_sheet.shape
print'Done'

rev_food_map = {v:k for k,v in food_map.items()}
k=20
a = ''
buf = []
with open('predict.csv', 'w') as f:
    f.write('userid,foodid\n')
    for i,user in enumerate(u_map.keys()):
        s = ''
        for food_idx in reversed(np.argsort(answer_sheet[i,:])[-k:]):
            s += ' %d' % rev_food_map[food_idx]
        f.write('%d,%s\n' % (user, s) )
        buf.append(a == s)
        a = s
print buf
print 'done'

    
            


In [ ]:
from constants import MAX_SEQ_LEN

print 'V2'        
flip = True       
model = torch.load('./best_v2.pt')
model.eval()
batch_size = 256
food_buf = torch.LongTensor(np.arange(len(food_map))).view(1,-1).repeat(batch_size, 1).cuda()
answer_sheet = np.zeros([len(u_map), len(food_map)])

with torch.no_grad():
    with tqdm(total=len(u_map)*len(food_map)) as pbar:
        keys = u_map.keys()
        ran = range((len(keys) // batch_size) +1) if len(keys) % batch_size != 0 else range(len(keys) // batch_size)
        for i in ran:
            a = i*batch_size
            b = (i+1)*batch_size if (i+1)*batch_size < len(keys) else len(keys)
            X = []
            for user in keys[a:b]:
                x = np.zeros([MAX_SEQ_LEN, len(food_map)])
                history = u_map[user]
                ds = np.array([d for d,f in history])
                fs = np.array([f for d,f in history])
                sorted_idx = np.flip(np.argsort(ds), axis=-1) if flip else np.argsort(ds)
                ds = ds[sorted_idx]
                fd = fs[sorted_idx]

                date_idx = 0
                now_date = ds[0]
                for food, date in zip(fs,ds):
                    if date != now_date:
                        date_idx+=1
                        now_date = date
                    x[date_idx, food_map[food]] = 1
                X.append(np.expand_dims(x,axis=0))
            
            X = torch.FloatTensor(np.vstack(X)).cuda()
            for food_idx  in range(len(food_map)):
                target = food_buf[:b-a, food_idx:food_idx+1]
                output = model(target, X)
                answer_sheet[a:b, food_idx:food_idx+1] =  output.cpu()
                pbar.update(batch_size)
                
np.save('./output_sheet2', answer_sheet)
print answer_sheet.shape
print'Done'

rev_food_map = {v:k for k,v in food_map.items()}
k=20
a = ''
buf = []
with open('predict2.csv', 'w') as f:
    f.write('userid,foodid\n')
    for i,user in enumerate(u_map.keys()):
        s = ''
        for food_idx in reversed(np.argsort(answer_sheet[i,:])[-k:]):
            s += ' %d' % rev_food_map[food_idx]
        f.write('%d,%s\n' % (user, s) )
        buf.append(a == s)
        a = s
print buf
print 'done'

    
            


In [ ]:
i = 0 
a = [b for b in reversed(np.argsort(answer_sheet[i,:])[-k:])]
print a

# i=1
a = [b for b in reversed(np.argsort(answer_sheet[i,:])[-k:])]
print a



In [ ]:
print answer_sheet[0,:]
print answer_sheet[1,:]

In [ ]:
# print acc_q
# print output.shape
# c = output < 0.5
# print output < 0.5

print pred.shape
print label.shape
print torch.sum(pred == label.type(torch.uint8)).item() / float(output.shape[0])

In [ ]:
a = torch.tensor([[0.49]])
b = torch.zeros_like(a)
print a.shape
print a > 0.5
c = a > 0.5
print b,c
print b.type(torch.uint8) == c


In [ ]:
salnj;kvahjk
# print u_map[6]
buf  = [] 
def ck(ds):
    l = max(ds) - min(ds)
    min_d = min(ds)
    
#     d_list = [(d-min_d) for d in ds]
#     d_list = sorted(set(d_list))
    d_list = sorted(set(ds))
#     print l, len(d_list)
    for i,d in enumerate(d_list):
        if i == 0:
            continue
        d_ = d_list[i-1]
        if (d - d_).days.real != 1:
            d_diff = (d - d_).days.real
            buf.append(d_diff)
#             print d_diff
#             assert d_diff < 10
def cf(ds):
    l = max(ds) - min(ds)
    min_d = min(ds)
    
#     d_list = [(d-min_d) for d in ds]
#     d_list = sorted(set(d_list))
    d_list = sorted(set(ds))
    
#     print l, len(d_list)
    for i,d in enumerate(d_list):
        if i == 0:
            continue
        d_ = d_list[i-1]
        if (d - d_).days.real != 1:
            d_diff = (d - d_).days.real
            buf.append(d_diff)
#             print d_diff
#             assert d_diff < 10
                        
    
# for i in [6,8,12]:
with tqdm(total=len(u_map)) as pbar:
    buf = []
    buf2 = []
    for i in u_map.keys():
        ds = [d for d,f in u_map[i]]
        threshold = ((max(ds) - min(ds)).days.real * (4./5.))
        min_d = min(ds)
        fl = []
        m = {}
        for d,f in u_map[i]:
            if (d - min_d).days.real > threshold:
                fl.append(f)
            if d not in m:
                m[d] = []
            m[d].append(f)
        for k,v in m.items():    
            buf2.append(len(set(v)))
#         print len(u_map[i]), len(fl), len(set(fl))
        buf.append(len(set(fl)))
        #     print i, min(ds), max(ds), len(ds)
#         ck(ds)
        pbar.update(1)
# d6 = [d for d,f in u_map[6]]
# ck(d6)
    
#     print ck(ds)
# print u_map[33]
print np.mean(buf), np.std(buf)
print np.mean(buf2), np.std(buf2)


In [ ]:


print min(dates), max(dates),  max(dates)- min(dates)


In [ ]:
print min(dates)
m = min(dates)
print max(dates)
a = dates[0]
print dir(a - min(dates))
print a
# def normalize_date(min_date, max_date, date):
# for k in u_map.keys()[:10]:
#     print len(u_map[k])
ds = [(d-m).total_seconds() / (60*60*24) for d in dates]
# import numpy as np
print np.mean(ds)
print np.min(ds), np.max(ds)

In [ ]:
import keras
from os.path import join
import os
from bs4 import BeautifulSoup as BS
from constants import MAX_SEQUENCE_LENGTH, MAX_NUM_WORDS, EMBEDDING_DIM
from keras.preprocessing.sequence import pad_sequences
import numpy as np
np.random.seed(1337)
def quote_title_abstract(xml_path):
    with open(xml_path, 'r') as f:
        data = f.read()
    soup = BS(data)
    title, abstract = soup.find('title').text, soup.find('abstract').text
    return title.strip(), abstract.strip()

# text preprocessing
data_path = join('./','kaggle/')
xml_dir = join(data_path, 't2-doc')
xml_list = [f for f in os.listdir(xml_dir) if f.endswith('.xml')]
# print(len(xml_list))


texts = []

for xml in xml_list:
    path = join(xml_dir,xml)
    title, abstract = quote_title_abstract(path)
    text = title + '' + abstract
    texts.append(text)
#     texts.append(title)
#     texts.append(abstract)
print('read all %d xml files.' % len(xml_list))
tokenizer = keras.preprocessing.text.Tokenizer(num_words=MAX_NUM_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~ ',
                                   lower=True, split=' ', char_level=False, oov_token=None)
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
sequences = tokenizer.texts_to_sequences(texts)
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
xml_id_map = {}
for i,xml in enumerate(xml_list):
    node_id = int(xml.replace('.xml',''))
    xml_id_map[node_id] = data[i,:]


print('Preparing embedding matrix.')
embeddings_index = {}
# with open(os.path.join('./','glove', 'glove.6B.%dd.txt' % EMBEDDING_DIM), 'r', encoding='utf8') as f:
with open(os.path.join('./','glove', 'glove.6B.%dd.txt' % EMBEDDING_DIM), 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print('done')